<a href="https://colab.research.google.com/github/MissoumYoucef/Kaggle_Competition/blob/main/Facial_Keypoints_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'facial-keypoints-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3486%2F31310%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240331%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240331T200152Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6322bc37c6d45d7967ec5131835b5b95b8aea731dbe881b29ee56d60b9cc8758e104812e89fd478e64f78f9d1d2af6e0ebe0c1b5a2cd85f991653786e07881eef38a1f156c178eda07f8dfb0e1e8d6db814cb04ddce33e49fe4ef9eb5cf6dc146cac11d3329b4e7e81d15ac914b2ef6bf5e4faf3eac550e1405d000c2a5e82cb9cbf9281d87feb0d719a1c5ecc93e304b8de12e1001d582d0294751d0b7b539c6f3f0913e2b5a057a3c6106dd0efb863b611651f75890ac5397febd7b8c915b523de04e3d724093c925f492c37c3837b84c017479021725ca3894cd3702de97a6db3c3fc25e6efc1b96b4ed70e0f9b002c67744b90133a3c9e14c68918d914ab'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
from keras.layers import Conv2D,Dropout,Dense,Flatten
from keras.models import Sequential
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

2024-03-31 05:03:55.483877: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 05:03:55.484045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 05:03:55.664067: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
train_data = pd.read_csv('/kaggle/input/facial-keypoints-detection/training.zip')
test_data = pd.read_csv('/kaggle/input/facial-keypoints-detection/test.zip')

In [ ]:
train_data.shape,test_data.shape

((7049, 31), (1783, 2))

In [ ]:
# Extract the image data and keypoint coordinates
images = train_data['Image'].apply(lambda x: np.array(x.split(' '), dtype=int))

In [ ]:
null_counts=train_data.isna().sum()
# null_counts

In [ ]:
non_much_Null_values=null_counts.index
non_much_Null_values = [col for col in non_much_Null_values if col != 'Image']
# non_much_Null_values

In [ ]:
for feature in non_much_Null_values:
    median_value = train_data[feature].median()
    train_data[feature].fillna(median_value, inplace=True)

/tmp/ipykernel_33/1613402553.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data[feature].fillna(median_value, inplace=True)
/tmp/ipykernel_33/1613402553.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [ ]:
keypoints = train_data.drop('Image', axis=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, keypoints, test_size=0.2, random_state=42)

In [ ]:
# Reshape the image data
X_train = np.array([x.reshape(96, 96, 1) for x in X_train])
X_val = np.array([x.reshape(96, 96, 1) for x in X_val])

In [ ]:
X_train.shape

(5639, 96, 96, 1)

In [ ]:
import tensorflow as tf
tf_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
# Assuming X_train is your grayscale image dataset with shape (num_images, 96, 96, 1)
X_train_rgb = tf.image.grayscale_to_rgb(tf_tensor)

In [ ]:
X_train_rgb.shape

TensorShape([5639, 96, 96, 3])

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2S,ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
# base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
# base_model.trainable = False  # Freeze the base model

In [ ]:
from tensorflow.keras.applications import ResNet50, EfficientNetB4, MobileNetV3Large

In [ ]:
from transformers import ViTFeatureExtractor, TFViTForImageClassification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load pretrained ViT feature extractor and model
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
# # vit_model = TFViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
feature_extractor.trainable = False
# # Freeze the base ViT model
# vit_model.trainable = False

In [ ]:
# Load ViT model
efficientnet_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
for layer in efficientnet_model.layers:
    layer.trainable = False

In [ ]:
resnext_model = ResNet50(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
for layer in resnext_model.layers:
    layer.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
def trainfun(model):
    model = models.Sequential([
        model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(30)  # Assuming a binary classification problem
    ])
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])
    return model

In [ ]:
model=trainfun(feature_extractor)
model.fit(X_train_rgb,y_train,epochs =5,batch_size = 256,validation_split = 0.2)

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  },
  "trainable": false
}
 (of type <class 'transformers.models.vit.feature_extraction_vit.ViTFeatureExtractor'>)

In [ ]:
model=trainfun(resnext_model)
model.fit(X_train_rgb,y_train,epochs =5,batch_size = 256,validation_split = 0.2)

Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 103s 5s/step - loss: 1543.6975 - mae: 33.5313 - val_loss: 220.3893 - val_mae: 10.1959
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 139s 5s/step - loss: 177.8687 - mae: 10.1276 - val_loss: 71.6242 - val_mae: 6.6209
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 90s 5s/step - loss: 81.9135 - mae: 6.9807 - val_loss: 41.5404 - val_mae: 4.8658
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - loss: 55.4235 - mae: 5.6732 - val_loss: 29.9716 - val_mae: 3.9950
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - loss: 44.6103 - mae: 5.0708 - val_loss: 25.4757 - val_mae: 3.6516


In [ ]:
model2=trainfun(efficientnet_model)
model2.fit(X_train_rgb,y_train,epochs =5,batch_size = 256,validation_split = 0.2)

Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 151s 7s/step - loss: 2154.3564 - mae: 42.3321 - val_loss: 390.4900 - val_mae: 15.4422
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 135s 7s/step - loss: 271.0721 - mae: 13.0111 - val_loss: 114.3324 - val_mae: 8.2508
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - loss: 93.6041 - mae: 7.4725 - val_loss: 50.3487 - val_mae: 5.0173
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 118s 7s/step - loss: 55.6354 - mae: 5.3509 - val_loss: 37.5856 - val_mae: 4.2466
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 146s 7s/step - loss: 44.4986 - mae: 4.6798 - val_loss: 30.1366 - val_mae: 3.8387


In [ ]:
#preparing test data
timag = []
for i in range(0,1783):
    timg = test_data['Image'][i].split(' ')
    timg = ['0' if x == '' else x for x in timg]

    timag.append(timg)

In [ ]:
timage_list = np.array(timag,dtype = 'float')
X_test = timage_list.reshape(-1,96,96,1)

In [ ]:
X_test.shape

(1783, 96, 96, 1)

In [ ]:
tf_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
# Assuming X_train is your grayscale image dataset with shape (num_images, 96, 96, 1)
X_test = tf.image.grayscale_to_rgb(tf_tensor)

In [ ]:
pred1 = model.predict(X_test)
pred2 = model2.predict(X_test)

56/56 ━━━━━━━━━━━━━━━━━━━━ 32s 525ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 43s 665ms/step


In [ ]:
pred = (pred1+pred2)/2

In [ ]:
lookid_data = pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)
rowid = lookid_data['RowId']
rowid=list(rowid)
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])
rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(preded,name = 'Location')
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('/kaggle/working/face_key_detection_submission.csv',index = False)

In [ ]:
fd=pd.read_csv('/kaggle/working/face_key_detection_submission.csv')

In [ ]:
fd[fd['Location'] > 96]

,RowId,Location


In [ ]:
fd.loc[fd['Location'] > 96, 'Location'] = 95
submission = fd

In [ ]:
submission.to_csv('/kaggle/working/face_key_detection_submission.csv',index = False)